einleitung
gesetzestexte ai act ...
forschungsfragen
ausarbeitung
fazit

todo: save methode, mehrere durchläufe?

## Modules und Imports ##

### Versionsliste der genutzten Modules ###

In [30]:
%pip list

Package                                 Version
--------------------------------------- ------------
aiofiles                                24.1.0
aiohappyeyeballs                        2.4.3
aiohttp                                 3.10.10
aiosignal                               1.3.1
annotated-types                         0.7.0
anyio                                   4.6.2.post1
asttokens                               2.4.1
attrs                                   24.2.0
backoff                                 2.2.1
beautifulsoup4                          4.12.3
certifi                                 2024.8.30
cffi                                    1.17.1
chardet                                 5.2.0
charset-normalizer                      3.4.0
click                                   8.1.7
colorama                                0.4.6
comm                                    0.2.2
cryptography                            44.0.0
dataclasses-json                        0.6.7
debugpy 

### Installation von benötigten Modules ###

In [31]:
#%pip install llama_index
#%pip install llama-index-llms-ollama
#%pip install llama-index-embeddings-huggingface

### Import der benötigten Modules ###

In [32]:
import os
from datetime import datetime

import base64
import gc
import random
import tempfile
import time
import uuid

from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader


from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

from llama_index.core import SimpleDirectoryReader

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, Document

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter

import re

from llama_index.core.schema import TextNode

import pandas as pd
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

from sentence_transformers import SentenceTransformer, util



## Definition benötigter Funktionen ##

### Einlesen der Fragen und Dokumente für die zusätzlichen Informationen im RAG-System ###

Definition der Funktion zum Import der zu beantwortenden Fragen.

Für jedes Modell, welches evaluiert werden soll, ist bereits eine Spalte in der Tabelle vorbereitet.

In [33]:
def load_questions(input_file_path:str = "D:/LLama3/questions.csv"):
    df = pd.read_csv(input_file_path, sep=";", encoding='latin-1')
    return df

Definition der Funktion zum Import der Dokumente, welche zusätzliche Daten zur Beantwortung der Fragen beinhalten. Alle in .txt-Form.

Kleine Dokumente enthalten:
- die ersten 7 Artikel des Grundgesetzes in jeweils einem Dokument
- Notwehrparagraph
- Statement aus dem Transferbericht (Test-Dokument)
- 2 kleine Fallbeispiele (Rakete & Superbahn) (Test-Dokumente)

Größere Dokumente enthalten:
- Ausschnitt aus Wikipedia zur Liste der Artikel im Grundgesetz
- Ausschnitt aus Wikipedia zum Grundgesetz
- Gesamtes Grundgesetz in Textform

In [34]:
def load_documents(input_dir_path: str = "D:/LLama3/documents/"):
    loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".txt"],
            recursive=True
        )
    return loader.load_data()

### Beantwortung der Frage durch das LLM ###

Definition der Funktion zur Beantwortung der Frage unter Berücksichtigung der mitgelieferten Antworten, welche als zusätzliche Kontext-Informationen genutzt werden sollen.

- System-Prompt mit grober Aufgabenstellung
- User-Prompt mit eingebauter Frage und hinzugefügtem Kontext

In [35]:
def answer_user_question(useful_answers: list, index: VectorStoreIndex, question: str):
    #System-Message
    qa_chat_instr_new = (
                """               
                    Beantworte die Frage mit Hilfe der Informationen aus dem hinzugefügten Kontext.\n
                """
    )
    #User-Message
    qa_prompt_tmpl_str_new = (
                """
                Kontext:
                {retrieved_graded_documents}\n

                Frage: 
                {query_str}\n"""
                )
    #Zusammenfügen aller mitgelieferten Antworten, sodass diese in der User-Message enthalten sind.
    if len(useful_answers) > 0:
        concat_answers = ""
        for answ in useful_answers:
            concat_answers = concat_answers + answ + "\n\n"

        qa_chat_instr_new_formatted = str(qa_chat_instr_new).format(
            retrieved_graded_documents=concat_answers#.text#, query_str=question
        )
        qa_prompt_tmpl_str_new_formatted = str(qa_prompt_tmpl_str_new).format(
            query_str=question,
            retrieved_graded_documents=concat_answers#.text#, query_str=question
        )
    else:
        qa_chat_instr_new_formatted = str(qa_chat_instr_new).format(
            retrieved_graded_documents=""
        )

        qa_prompt_tmpl_str_new_formatted = str(qa_prompt_tmpl_str_new).format(
            query_str=question,
            retrieved_graded_documents=""#.text#, query_str=question

        )

    #Setup der System- und User-Message für das LLM
    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=(
                qa_chat_instr_new_formatted
            ),
        ),
        ChatMessage(role=MessageRole.USER, content=qa_prompt_tmpl_str_new_formatted),
    ]
    
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
    #Setup der Query-Engine für die Beantwortung der Frage
    query_engine_evaluator = index.as_query_engine(similarity_top_k=4, text_qa_template=text_qa_template)
    #Beantwortung der Frage auf Basis der zuvor festgelegten Parameter und Informationen
    result = query_engine_evaluator.query(question)
    #print(result)
    return result

### Abruf von Dokumenten aus Vector Store ###

Definition der Funktion zum Abruf der top_k Dokumente mit einer Similarity zur Fragestellung von mindestens 0.5 aus dem Vector-Store. 

In [36]:
def retrieve_documents_from_vectorstore_new(question: str, index: VectorStoreIndex):
    top_k = 4

    # Retriever konfigurieren
    retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
    )

    # Query Engine definieren
    query_engine = RetrieverQueryEngine(        
        retriever=retriever,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
    )
    #Query Engine mit Frage nach Dokumenten suchen lassen
    response = query_engine.query(question)
    # Antwort in gewünschte Form bringen
    retr_answers = []
    #for i in range(top_k):
    for i in range(len(response.source_nodes)):
        retr_answers.append(response.source_nodes[i].text)
        
    return retr_answers

Anderer Ansatz zum Abrufen der Dokumente aus anderer Art von Doc-Store. Wird aktuell nicht genutzt und KANN (wahrscheinlich) WEG.

In [37]:
def retrieve_documents_from_vectorstore_new2(question: str, index_base_docs: list, embedding_model: SentenceTransformer, doc_emb):
    newtopk = 4

    #Encode query and documents
    #print("embedding question and docs")
    query_emb = embedding_model.encode(question)
    

    #Compute dot score between query and all document embeddings
    #print("computing scores")
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

    #Combine docs & scores
    #print("combining docs & scores")
    doc_score_pairs = list(zip(index_base_docs, scores))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

    #Output passages & scores
    #for doc, score in doc_score_pairs:
    #    print(score, doc)

    top_k_doc_score_pairs = doc_score_pairs[:newtopk]

    top_k_docs = []
    for doc_score in top_k_doc_score_pairs:
        top_k_docs.append(doc_score[0].text)

    #print(top_k_docs[0])
    return top_k_docs

### Bewertung von abgerufenen Dokumenten im Hinblick auf Fragestellung ###

In [38]:
def grade_retrieved_documents_new(question:str, retr_answers: list, index: VectorStoreIndex):
    #System-Message
    doc_grader_instr_new = """Du bewertest die Relevanz von einem abgerufenen Dokument für die Frage eines Benutzers. 
    Bewerte sorgfältig und objektiv, ob das Dokument Stichworte oder Informationen enthält, die einen Bezug zu der Frage haben.
    Gib eine Antwort im JSON Format, welche den Wert 'ja' oder 'nein' enthält.
    Wenn das Dokument Stichworte oder Informationen mit Bezug zur Frage enthält, antworte mit 'ja', sonst mit 'nein'.
    """

    #User-Message
    doc_grader_prompt_new = """Abgerufenes Dokument: \n\n {document} \n\n. Frage: \n\n {question}."""
    

    #Da die Antworten nicht immer tatsächlich relevant sind bzw. Hallucinations beinhalten müssen diese erneut evaluiert werden. 
    #Hierfür wird dem LLM gesagt, dass es die erhaltenen Dokumente hinsichtlich der Fragestellung erneut bewerten soll.
    useful_answers = []
    for answer in retr_answers:
        #print(doc.text)
        doc_grader_prompt_formatted = str(doc_grader_prompt_new).format(
        document=answer, question=question
        )

        # Text QA Prompt
        chat_text_qa_msgs = [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=(
                    doc_grader_instr_new
                ),
            ),
            ChatMessage(role=MessageRole.USER, content=doc_grader_prompt_formatted),
        ]
        
        text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

        query_engine_doc_grader = index.as_query_engine(similarity_top_k=3, text_qa_template=text_qa_template)

        #Auswahl anhand der Antwort, sodass nur "nützliche" Antworten weiter verarbeitet werden
        result = query_engine_doc_grader.query(question)
        if 'ja' in str(result).lower():
            useful_answers.append(answer)
            print(result)
            print(answer)


    return(useful_answers)


## Durchführung des Modell-Vergleichs ## 

### Dokumente und Fragen aus Verzeichnissen laden ###

In [39]:
questions = load_questions()
docs = load_documents()
#Für Dateiname am Ende
currtime = time.strftime("%Y_%m_%d__%H_%M_%S")


### Vorbereitung bestimmter Dokumente für Indexierung ###

Ausgewählte Dokumente werden in ihre einzelnen Artikel aufgeteilt, um semantisch zusammenhängende Dokumente zu erstellen. Hierbei ist anschließend ein Artikel einem Dokument gleichzusetzen.

In [40]:
#Benötigte Variablen vorbereiten
text_chunks= []
index_base_docs = []

#Bestimmte Dokumente gesondert aufbereiten 
for doc in docs:
    if "Grundgesetz - Kopie.txt" in doc.metadata["file_path"] or "Grundgesetz.txt" in doc.metadata["file_path"]:
        #Regex für die spezifischen Dokumente zum Trennen der Inhalte/Artikel
        text_chunks = re.split('(?s)(\nArtikel [0-9]+.*?(?=\nArtikel))', doc.text)
        for chunk in text_chunks:
            if not(chunk == "" or not chunk):               
                index_base_docs.append(Document(text=chunk))
    elif "Grundgesetz2.txt" in doc.metadata["file_path"]:
        #Regex für die spezifischen Dokumente zum Trennen der Inhalte/Artikel
        text_chunks = re.split('(?s)(.*?)(?=\nArt )(\nArt [0-9]+.*?(?=\nArt ))', doc.text)
        for chunk in text_chunks:
            if not(chunk == "" or not chunk):
                index_base_docs.append(Document(text=chunk))
    else:
        index_base_docs.append(doc)

#Entfernen von unnötigen tab, newline, whitespaces
for doc in index_base_docs:
    doc.text = str(re.sub(r"\s+", " ", doc.text))



### Setup des Embedding Models und Durchführung der Indexierung  ###

In [41]:
#Zu Beginn genutzte Modelle mit allgemeiner Verwendung

#embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/all-MiniLM-L6-v2", trust_remote_code=True)
##embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1", trust_remote_code=True)
#embed_model = HuggingFaceEmbedding( model_name="nomic-ai/nomic-embed-text-v1", trust_remote_code=True)


#Spezifische für Asymmetrische Semantic Search trainierte Embedding Models

#embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/msmarco-distilbert-base-tas-b", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/msmarco-bert-base-dot-v5", trust_remote_code=True)

#Festlegen des verwendeten Embedding Models
Settings.embed_model = embed_model

#Definition anzuwendender Transformationen für die eingelesenen Dokumente
transformations=[
        SentenceSplitter(chunk_size=256, chunk_overlap=10),
        #TitleExtractor(),
        #OpenAIEmbedding(),
    ]
#Erstellung des VectorStoreIndex für die Dokumente unter Berücksichtigung der Transformationen
index = VectorStoreIndex.from_documents(index_base_docs, show_progress=True, transformations=transformations)

Generating embeddings: 100%|██████████| 1730/1730 [07:43<00:00,  3.73it/s]


### Vergleich der LLama Models ###

Liste der zu vergleichenden Modelle

In [42]:
#Auslesen der Modelle aus Question-Dataframe
questions_models = pd.DataFrame(questions.columns.values).tail(-1)
for questions_models_index, questions_model in questions_models.iterrows():
    print(questions_model[0])

llama3.2:1b
llama3.2:3b
llama3.1:8b


Durchführung der Beantwortung der Fragen durch die einzelnen Modelle

In [43]:
#Vorbereitung der Analyse-Variable
answers_new = []

iterations = 3
for i in range(iterations):
    #Iteration durch alle Modelle und anschließend für jedes Modell durch alle Fragen
    for questions_models_index, questions_model in questions_models.iterrows():
        current_model = questions_model[0]
        print("Question Model Index: " + str(questions_models_index) + " Model: " + str(current_model))
        
        # Festlegen des aktuellen LLMs sodass dieses verwendet wird
        # json-mode oder default-mode wird je nach Funktion ausgewählt
        llm = Ollama(model=current_model, request_timeout=120.0)
        llm_json = Ollama(model=current_model, request_timeout=120.0, json_mode=True) 
        Settings.llm = llm 

        #durch alle Fragen iterieren und diese anhand des zuvor festgelegten Ablaufs beantworten
        for questions_index, questions_row in questions.iterrows():
            print("Question Index: " + str(questions_index))
            df_question = str(questions_row['Frage'])
            #Dokumente aus VectorStoreIndex laden
            retrieved_answers_new = retrieve_documents_from_vectorstore_new(df_question, index)
            print("Retrieved Answers: " + str(len(retrieved_answers_new)))

            #Bewertung der erhaltenen Dokumente hinsichtlich tatsächlicher Relevanz
            Settings.llm = llm_json 
            useful_answers_new = grade_retrieved_documents_new(df_question, retrieved_answers_new, index)
            print("Useful Answers: " + str(len(useful_answers_new)))

            Settings.llm = llm 
            #Beantwortung der Frage auf Grundlage der mitgelieferten Kontext-Informationen
            answer_new = str(answer_user_question(useful_answers=useful_answers_new, index=index, question=df_question))

            answers_new.append(answer_new)
            questions.iloc[questions_index, questions_models_index] = answer_new
            
            #Export results to Filename
            questions_df = pd.DataFrame(questions)
            export_filename = currtime + "_questions_filled_" + str(i) + ".csv"
            questions_df.to_csv("D:/LLama3/results/" + export_filename, sep=";", encoding='latin-1', index=False)
            #questions_df.to_csv()


Question Model Index: 1 Model: llama3.2:1b
Question Index: 0
Retrieved Answers: 4
{
  "dokument": {
    "title": "Strafgesetzbuch Allgemeiner Teil (§§ 1 - 79b)",
    "content": "§ 32 Notwehr (1) Wer eine Tat begeht, die durch Notwehr geboten ist, handelt nicht rechtswidrig. (2) Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.",
    "relevance": "ja"
  }
}
Strafgesetzbuch Allgemeiner Teil (§§ 1 - 79b) 2. Abschnitt - Die Tat (§§ 13 - 37) 4. Titel - Notwehr und Notstand (§§ 32 - 35) § 32 Notwehr (1) Wer eine Tat begeht, die durch Notwehr geboten ist, handelt nicht rechtswidrig. (2) Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.
{
  " Antwort": "ja"
}
(5) Das Recht des öffentlichen Dienstes ist unter Berücksichtigung der hergebrachten Grundsätze des Berufsbeamtentums zu regeln und fortzuentwickeln.
{
  "richterli

C:\Users\doein\AppData\Local\Temp\ipykernel_19716\1467948926.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  questions.iloc[questions_index, questions_models_index] = answer_new


Question Index: 1
Retrieved Answers: 3
{
  "Antwort": "ja",
  "Details": {
    "Referenz": "Artikel 2 (1) Bundesverfassungsgesetz",
    "Stichworte/Informationen": [
      "Rechtsgrundlage für Verfassungsvereinbarungen, Grundrechte des Einzelnen"
    ]
  }
}
Artikel 83 Die Länder führen die Bundesgesetze als eigene Angelegenheit aus, soweit dieses Grundgesetz nichts anderes bestimmt oder zuläßt.
{
  "antwort": "ja",
  "dokument": {
    "titel": "Artikel 2 (1) und (2) des deutschen Grundgesetzes"
  }
}
Artikel 2 (1) Jeder hat das Recht auf die freie Entfaltung seiner Persönlichkeit, soweit er nicht die Rechte anderer verletzt und nicht gegen die verfassungsmaßige Ordnung oder das Sittengesetz verstößt. (2) Jeder hat das Recht auf Leben und körperliche Unversehrtheit. Die Freiheit der Person ist unverletzlich. In diese Rechte darf nur auf Grund eines Gesetzes eingegriffen werden.
Useful Answers: 2
Question Index: 2
Retrieved Answers: 3
Useful Answers: 0
Question Index: 3
Retrieved Answer

C:\Users\doein\AppData\Local\Temp\ipykernel_19716\1467948926.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  questions.iloc[questions_index, questions_models_index] = answer_new


Question Index: 1
Retrieved Answers: 3
{
  "Relevanz": "ja"
}
Artikel 2 (1) Jeder hat das Recht auf die freie Entfaltung seiner Persönlichkeit, soweit er nicht die Rechte anderer verletzt und nicht gegen die verfassungsmaßige Ordnung oder das Sittengesetz verstößt. (2) Jeder hat das Recht auf Leben und körperliche Unversehrtheit. Die Freiheit der Person ist unverletzlich. In diese Rechte darf nur auf Grund eines Gesetzes eingegriffen werden.
Useful Answers: 1
Question Index: 2
Retrieved Answers: 3
{
  "Relevant": "ja"
}
Mein Statement zum Abschlusskolloquium: Der erste Transferbericht zeigt, wie verschiedene Themengebiete aus dem Cloud-Umfeld, welches für die Big Data Architektur genutzt wird, auf konventionelle, nicht Cloud-native Anwendungen angewendet werden kann und dort zu Vorteilen und Nutzen führen kann. Ebenso können die unterrichteten Prinzipien dazu genutzt werden, um im Rahmen der Trans-formation auf Vor- und Nachteile gewisser Strukturen hinzuweisen und somit den Entschei-d

C:\Users\doein\AppData\Local\Temp\ipykernel_19716\1467948926.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Die Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  questions.iloc[questions_index, questions_models_index] = answer_new


Question Index: 1
Retrieved Answers: 3
{
  "relevant": "ja"
}
Artikel 2 (1) Jeder hat das Recht auf die freie Entfaltung seiner Persönlichkeit, soweit er nicht die Rechte anderer verletzt und nicht gegen die verfassungsmaßige Ordnung oder das Sittengesetz verstößt. (2) Jeder hat das Recht auf Leben und körperliche Unversehrtheit. Die Freiheit der Person ist unverletzlich. In diese Rechte darf nur auf Grund eines Gesetzes eingegriffen werden.
Useful Answers: 1
Question Index: 2
Retrieved Answers: 3
{"relevant": "ja"}
Mein Statement zum Abschlusskolloquium: Der erste Transferbericht zeigt, wie verschiedene Themengebiete aus dem Cloud-Umfeld, welches für die Big Data Architektur genutzt wird, auf konventionelle, nicht Cloud-native Anwendungen angewendet werden kann und dort zu Vorteilen und Nutzen führen kann. Ebenso können die unterrichteten Prinzipien dazu genutzt werden, um im Rahmen der Trans-formation auf Vor- und Nachteile gewisser Strukturen hinzuweisen und somit den Entschei-dungs